In [ ]:
import os
import re
from typing import Optional
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from jira import JIRA

class JiraTicketTranslator:
    """Jira 티켓을 번역하면서 이미지/첨부파일 마크업을 유지하는 클래스"""
    
    def __init__(self, jira_url: str, email: str, api_token: str, openai_api_key: str):
        """
        Args:
            jira_url: Jira 인스턴스 URL (예: 'https://cloud.jira.krafton.com')
            email: Jira 계정 이메일
            api_token: Jira API 토큰
            openai_api_key: OpenAI API 키
        """
        # Jira 클라이언트 초기화
        self.jira = JIRA(
            server=jira_url,
            basic_auth=(email, api_token)
        )
        
        # LangChain LLM 초기화
        self.llm = ChatOpenAI(
            model="gpt-4o",
            api_key=openai_api_key,
            temperature=0
        )
        
        # 번역 프롬프트 템플릿
        self.translation_prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a professional translator. Translate the following text to {target_language}. "
                      "Preserve any Jira markup syntax like *bold*, _italic_, {{code}}, etc. "
                      "Only translate the actual text content, not the markup symbols."),
            ("user", "{text}")
        ])
    
    def extract_attachments_markup(self, text: str) -> tuple[list[str], str]:
        """
        Jira 마크업에서 이미지와 첨부파일 마크업을 추출하고 플레이스홀더로 대체
        
        Args:
            text: 원본 텍스트
            
        Returns:
            (마크업 리스트, 플레이스홀더가 적용된 텍스트)
        """
        if not text:
            return [], ""
        
        attachments = []
        
        # 이미지 마크업 패턴: !image.png!, !image.png|thumbnail!, !image.png|width=300!
        image_pattern = r'!([^!]+?)(?:\|[^!]*)?!'
        
        # 첨부파일 마크업 패턴: [^attachment.pdf], [^video.mp4]
        attachment_pattern = r'\[\^([^\]]+?)\]'
        
        def replace_image(match):
            attachments.append(match.group(0))
            return f"__IMAGE_PLACEHOLDER_{len(attachments)-1}__"
        
        def replace_attachment(match):
            attachments.append(match.group(0))
            return f"__ATTACHMENT_PLACEHOLDER_{len(attachments)-1}__"
        
        # 플레이스홀더로 대체
        text = re.sub(image_pattern, replace_image, text)
        text = re.sub(attachment_pattern, replace_attachment, text)
        
        return attachments, text
    
    def restore_attachments_markup(self, text: str, attachments: list[str]) -> str:
        """
        번역된 텍스트에 원본 마크업을 복원
        
        Args:
            text: 번역된 텍스트 (플레이스홀더 포함)
            attachments: 원본 마크업 리스트
            
        Returns:
            마크업이 복원된 텍스트
        """
        for i, attachment_markup in enumerate(attachments):
            # 이미지 플레이스홀더 복원
            text = text.replace(f"__IMAGE_PLACEHOLDER_{i}__", attachment_markup)
            # 첨부파일 플레이스홀더 복원
            text = text.replace(f"__ATTACHMENT_PLACEHOLDER_{i}__", attachment_markup)
        
        return text
    
    def translate_text(self, text: str, target_language: str = "Korean") -> str:
        """
        텍스트를 번역 (마크업 제외)
        
        Args:
            text: 번역할 텍스트
            target_language: 목표 언어
            
        Returns:
            번역된 텍스트
        """
        if not text or not text.strip():
            return text
        
        chain = self.translation_prompt | self.llm
        result = chain.invoke({
            "text": text,
            "target_language": target_language
        })
        
        return result.content
    
    def translate_field(self, field_value: str, target_language: str = "Korean") -> str:
        """
        Jira 필드 값을 번역 (이미지/첨부파일 마크업 보존)
        
        Args:
            field_value: 원본 필드 값
            target_language: 목표 언어
            
        Returns:
            번역된 필드 값 (마크업 보존)
        """
        if not field_value:
            return field_value
        
        # 1. 이미지/첨부파일 마크업 추출
        attachments, clean_text = self.extract_attachments_markup(field_value)
        
        # 2. 텍스트만 번역
        translated_text = self.translate_text(clean_text, target_language)
        
        # 3. 마크업 복원
        final_text = self.restore_attachments_markup(translated_text, attachments)
        
        return final_text
    
    def translate_issue(
        self, 
        issue_key: str, 
        target_language: str = "Korean",
        fields_to_translate: Optional[list[str]] = None
    ) -> dict:
        """
        Jira 이슈를 번역
        
        Args:
            issue_key: Jira 이슈 키 (예: 'BUG-123')
            target_language: 목표 언어
            fields_to_translate: 번역할 필드 리스트 (기본: ['summary', 'description'])
            
        Returns:
            번역 결과 딕셔너리
        """
        if fields_to_translate is None:
            fields_to_translate = ['summary', 'description']
        
        # 1. 이슈 조회
        print(f"📥 Fetching issue {issue_key}...")
        issue = self.jira.issue(issue_key)
        
        # 2. 각 필드 번역
        update_fields = {}
        translation_results = {}
        
        for field in fields_to_translate:
            field_value = getattr(issue.fields, field, None)
            
            if field_value:
                print(f"🔄 Translating {field}...")
                translated_value = self.translate_field(field_value, target_language)
                update_fields[field] = translated_value
                translation_results[field] = {
                    'original': field_value,
                    'translated': translated_value
                }
        
        # 3. 이슈 업데이트
        if update_fields:
            print(f"💾 Updating issue {issue_key}...")
            issue.update(fields=update_fields)
            print(f"✅ Successfully translated {issue_key}!")
        else:
            print(f"⚠️ No fields to translate for {issue_key}")
        
        return translation_results


# 사용 예시
if __name__ == "__main__":
    # 설정
    JIRA_URL = "https://cloud.jira.krafton.com"
    JIRA_EMAIL = os.getenv("JIRA_EMAIL")
    JIRA_API_TOKEN = os.getenv("JIRA_API_TOKEN")
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
    
    # 번역기 초기화
    translator = JiraTicketTranslator(
        jira_url=JIRA_URL,
        email=JIRA_EMAIL,
        api_token=JIRA_API_TOKEN,
        openai_api_key=OPENAI_API_KEY
    )
    
    # 티켓 번역 (summary와 description)
    issue_key = "BUG-123"
    results = translator.translate_issue(
        issue_key=issue_key,
        target_language="Korean",
        fields_to_translate=['summary', 'description']
    )
    
    # 결과 출력
    print("\n📊 Translation Results:")
    print("="*50)
    for field, content in results.items():
        print(f"\n{field.upper()}:")
        print(f"Original: {content['original'][:100]}...")
        print(f"Translated: {content['translated'][:100]}...")

: 

In [ ]:
#https://cloud.jira.krafton.com/browse/P2-70735


